El objetivo de este script es juntar la información proveniente de distintas tablas y resumirlo en una fila por cada cliente para dejar dos CSVs listos para realizar un modelo de predicción de las variables DiasPago y DiasCalle. Para tal fin se crean nuevas variables, particularmente una de "score" a fin de calificar la periodicidad de Pago

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Lectura de CSVs suministrados. Deben estar en la misma carpeta que este script o completar el path 
df_completa = pd.read_csv("Completa.csv", sep = ";", 
                          names = ["Clientes", "Date", "Importe_comp", "DiasCalle_comp", "DiasPago_comp"])

df_diascalle = pd.read_csv("Dias Calle decimal.csv", sep = ";", 
                          names = ["Clientes", "Date", "Importe_dc", "Importe_comprobante", "DiasCalle"])

df_diaspago = pd.read_csv("Dias Pago Decimal.csv", sep = ";", 
                          names = ["Clientes", "Date", "Importe_dp", "DiasPago", "Monto", "TipoPago"])

In [3]:
# Controlando total de filas, datatype y nulos
df_completa.info()
df_diascalle.info()
df_diaspago.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83440 entries, 0 to 83439
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Clientes        83440 non-null  int64 
 1   Date            83440 non-null  object
 2   Importe_comp    83440 non-null  int64 
 3   DiasCalle_comp  83440 non-null  int64 
 4   DiasPago_comp   83440 non-null  int64 
dtypes: int64(4), object(1)
memory usage: 3.2+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 252643 entries, 0 to 252642
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   Clientes             252643 non-null  int64  
 1   Date                 252643 non-null  object 
 2   Importe_dc           252643 non-null  float64
 3   Importe_comprobante  252643 non-null  float64
 4   DiasCalle            252643 non-null  int64  
dtypes: float64(2), int64(2), object(1)
memory usage: 9.6+ MB
<clas

In [4]:
#Controlando la correcta importación de columnas
df_completa.head(5)
df_diascalle.head(5)
df_diaspago.head(5)

,Clientes,Date,Importe_comp,DiasCalle_comp,DiasPago_comp
0,36,2022-08-04 00:00:00.000,87601,50,32
1,239,2022-08-04 00:00:00.000,20511,1,0
2,711,2022-08-04 00:00:00.000,993,1,0
3,827,2022-08-04 00:00:00.000,148944,26,45
4,866,2022-08-04 00:00:00.000,15980,8,44


,Clientes,Date,Importe_dc,Importe_comprobante,DiasCalle
0,1771,2014-09-12 00:00:00.000,451.0,80.0,87
1,6998,2014-08-26 00:00:00.000,816.0,816.0,35
2,2196,2014-10-15 00:00:00.000,1825.0,684.0,85
3,1541,2014-08-20 00:00:00.000,950.0,950.0,29
4,713,2014-08-28 00:00:00.000,301.0,301.0,37


,Clientes,Date,Importe_dp,DiasPago,Monto,TipoPago
0,110,2014-07-31 00:00:00.000,1548.0,39,2632.0,VAL
1,479,2014-07-31 00:00:00.000,7607.0,20,13800.0,VAL
2,2000,2014-08-04 00:00:00.000,2479.0,40,3413.0,VAL
3,487,2014-08-05 00:00:00.000,1323.0,35,3000.0,VAL
4,1250,2014-08-06 00:00:00.000,1088.0,-27,1650.0,VAL


In [5]:
#Cantidad de Clientes en cada CSV. Se usarán los de completa.csv como base.
len(df_completa["Clientes"].unique())
len(df_diascalle["Clientes"].unique())
len(df_diaspago["Clientes"].unique())

2446

2464

2447

In [6]:
#ELiminando aquellos valores negativos. Se puede analizar si se eliminan, se corrigen a cero o si se multipiclan por -1 
len(df_completa)
df_completa = df_completa[(df_completa["DiasCalle_comp"]>-1) & (df_completa["DiasPago_comp"]>-1) ]
len(df_completa)
len(df_diascalle)
df_diascalle = df_diascalle[df_diascalle["DiasCalle"]>-1]
len(df_diascalle)
len(df_diaspago)
df_diaspago = df_diaspago[df_diaspago["DiasPago"]>-1]
len(df_diaspago)

83440

82934

252643

246079

162084

149927

In [7]:
#Creando un id de cada transacción para realizar un merge controlado entre los 3 CSV. 
#Transaccion = Cada Cliente en cada Date
df_completa = df_completa.reset_index()
df_completa = df_completa.rename(columns = {"index": "ID_transaction"})
df_completa

,ID_transaction,Clientes,Date,Importe_comp,DiasCalle_comp,DiasPago_comp
0,0,36,2022-08-04 00:00:00.000,87601,50,32
1,1,239,2022-08-04 00:00:00.000,20511,1,0
2,2,711,2022-08-04 00:00:00.000,993,1,0
3,3,827,2022-08-04 00:00:00.000,148944,26,45
4,4,866,2022-08-04 00:00:00.000,15980,8,44
...,...,...,...,...,...,...
82929,83435,851,2014-07-23 00:00:00.000,580,0,0
82930,83436,2001,2014-07-23 00:00:00.000,1169,0,0
82931,83437,2261,2014-07-23 00:00:00.000,351,1,0
82932,83438,397,2014-07-22 00:00:00.000,183,0,0


In [8]:
#Merge con diaspago
transaction = df_completa[["Clientes", "Date", "ID_transaction"]]
df_diaspago = pd.merge(df_diaspago, transaction, how = "inner")
df_diaspago

,Clientes,Date,Importe_dp,DiasPago,Monto,TipoPago,ID_transaction
0,110,2014-07-31 00:00:00.000,1548.00,39,2632.00,VAL,83398
1,110,2014-07-31 00:00:00.000,1548.00,0,-1084.00,CAJ,83398
2,110,2014-07-31 00:00:00.000,1084.40,0,1084.40,CAJ,83398
3,2000,2014-08-04 00:00:00.000,2479.00,40,3413.00,VAL,83392
4,2000,2014-08-04 00:00:00.000,2479.00,0,-933.00,CAJ,83392
...,...,...,...,...,...,...,...
149077,2461,2022-08-02 00:00:00.000,2263.40,0,2263.40,CAJ,83
149078,2510,2022-08-02 00:00:00.000,501.53,0,501.53,CAJ,91
149079,2497,2022-08-02 00:00:00.000,1243.88,0,1243.88,CAJ,88
149080,2483,2022-08-02 00:00:00.000,1458.72,0,1458.72,CAJ,86


In [9]:
#Data historica de cada Cliente para DF final
data_clientes = df_completa.groupby(['Clientes'])
counts = data_clientes.size().to_frame(name='counts')
counts = (counts
   .join(data_clientes.agg({'DiasCalle_comp': 'mean'}).rename(columns={'DiasCalle_comp': 'DiasCalle_mean'}))
   .join(data_clientes.agg({'DiasPago_comp': 'mean'}).rename(columns={'DiasPago_comp': 'DiasPago_mean'}))
   .join(data_clientes.agg({'DiasPago_comp': 'max'}).rename(columns={'DiasPago_comp': 'DiasPago_max'}))
   .join(data_clientes.agg({'DiasPago_comp': 'min'}).rename(columns={'DiasPago_comp': 'DiasPago_min'}))
   .reset_index()
   )

In [10]:
counts

,Clientes,counts,DiasCalle_mean,DiasPago_mean,DiasPago_max,DiasPago_min
0,1,17,18.941176,0.000000,0,0
1,2,48,14.750000,0.000000,0,0
2,3,10,16.400000,3.300000,26,0
3,4,29,1.655172,0.000000,0,0
4,5,1,2.000000,0.000000,0,0
...,...,...,...,...,...,...
2434,10070,125,39.720000,15.920000,858,0
2435,100067,1,19.000000,0.000000,0,0
2436,100748,77,24.207792,5.051948,39,0
2437,100749,5,23.400000,12.600000,22,0


In [11]:
# Preparando data del último año. Completa
import warnings
warnings.filterwarnings('ignore')

df_completa["Date"] = pd.to_datetime(df_completa["Date"])
desde = datetime.today() - timedelta(days=550)
last_year = df_completa[df_completa["Date"]> desde]
last_year["year_dummy"] = 1
last_year

,ID_transaction,Clientes,Date,Importe_comp,DiasCalle_comp,DiasPago_comp,year_dummy
0,0,36,2022-08-04,87601,50,32,1
1,1,239,2022-08-04,20511,1,0,1
2,2,711,2022-08-04,993,1,0,1
3,3,827,2022-08-04,148944,26,45,1
4,4,866,2022-08-04,15980,8,44,1
...,...,...,...,...,...,...,...
23410,23544,2146,2021-02-19,8519,2,0,1
23411,23545,2210,2021-02-19,7287,1,0,1
23412,23546,2232,2021-02-19,5282,7,0,1
23413,23547,2252,2021-02-19,16937,33,0,1


In [12]:
#Sumando aquellos que no hayan tenido transacciones el último año
clientes_completa = list(last_year["Clientes"].unique())
clientes_completa = df_completa[~df_completa["Clientes"].isin(clientes_completa)]
clientes_completa["year_dummy"] = 0
last_year = pd.concat([last_year, clientes_completa]) 
last_year

,ID_transaction,Clientes,Date,Importe_comp,DiasCalle_comp,DiasPago_comp,year_dummy
0,0,36,2022-08-04,87601,50,32,1
1,1,239,2022-08-04,20511,1,0,1
2,2,711,2022-08-04,993,1,0,1
3,3,827,2022-08-04,148944,26,45,1
4,4,866,2022-08-04,15980,8,44,1
...,...,...,...,...,...,...,...
82921,83427,1975,2014-07-25,125,0,0,0
82925,83431,188,2014-07-24,912,0,0,0
82928,83434,417,2014-07-23,72,0,0,0
82931,83437,2261,2014-07-23,351,1,0,0


In [13]:
#Se dividen el promedio de Diaspago según Categoria para Score. Ignorar Warnings

last_year["menos_15"] = np.where(last_year["DiasPago_comp"]<15, 1 ,0)
last_year["menos_30"] = np.where(
    (last_year["DiasPago_comp"]>=15) & (last_year["DiasPago_comp"]<30), 1 ,0)
last_year["menos_45"] = np.where(
    (last_year["DiasPago_comp"]>=30) & (last_year["DiasPago_comp"]<45), 1 ,0)
last_year["menos_75"] = np.where(
    (last_year["DiasPago_comp"]>=45) & (last_year["DiasPago_comp"]<75), 1 ,0)
last_year["mas_75"] = np.where(last_year["DiasPago_comp"]>=75, 1 ,0)

last_year

,ID_transaction,Clientes,Date,Importe_comp,DiasCalle_comp,DiasPago_comp,year_dummy,menos_15,menos_30,menos_45,menos_75,mas_75
0,0,36,2022-08-04,87601,50,32,1,0,0,1,0,0
1,1,239,2022-08-04,20511,1,0,1,1,0,0,0,0
2,2,711,2022-08-04,993,1,0,1,1,0,0,0,0
3,3,827,2022-08-04,148944,26,45,1,0,0,0,1,0
4,4,866,2022-08-04,15980,8,44,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
82921,83427,1975,2014-07-25,125,0,0,0,1,0,0,0,0
82925,83431,188,2014-07-24,912,0,0,0,1,0,0,0,0
82928,83434,417,2014-07-23,72,0,0,0,1,0,0,0,0
82931,83437,2261,2014-07-23,351,1,0,0,1,0,0,0,0


In [14]:
#Se agrupan y suman para cada cliente
last_year = last_year.groupby("Clientes").sum().reset_index()
last_year

,Clientes,ID_transaction,Importe_comp,DiasCalle_comp,DiasPago_comp,year_dummy,menos_15,menos_30,menos_45,menos_75,mas_75
0,1,72953,149565,111,0,7,7,0,0,0,0
1,2,2999076,140191,708,0,0,48,0,0,0,0
2,3,740189,11811,164,33,0,9,1,0,0,0
3,4,2228301,47538,48,0,0,29,0,0,0,0
4,5,63749,982,2,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2434,10070,7594378,1690986,4965,1990,0,102,5,5,9,4
2435,100067,12354,428581,19,0,1,1,0,0,0,0
2436,100748,141755,207738,105,42,15,14,1,0,0,0
2437,100749,396556,8941,117,63,0,2,3,0,0,0


In [15]:
#Se calcula el Total de transacciones y el Score
last_year["Total"] = last_year[["menos_15", "menos_30", "menos_45", "menos_75", "mas_75"]].sum(axis=1)
last_year["Score"] = 4
last_year["Score"] = np.where((last_year["menos_15"]+last_year["menos_30"]+last_year["menos_45"])/
                              last_year["Total"]>0.9, 3, last_year["Score"])
last_year["Score"] = np.where((last_year["menos_15"]+last_year["menos_30"])/last_year["Total"]>0.95, 2, last_year["Score"])
last_year["Score"] = np.where(last_year["menos_15"]/last_year["Total"]>0.95, 1, last_year["Score"])
last_year["Score"] = np.where(last_year["mas_75"]>0, last_year["Score"]+1, last_year["Score"])
last_year

,Clientes,ID_transaction,Importe_comp,DiasCalle_comp,DiasPago_comp,year_dummy,menos_15,menos_30,menos_45,menos_75,mas_75,Total,Score
0,1,72953,149565,111,0,7,7,0,0,0,0,7,1
1,2,2999076,140191,708,0,0,48,0,0,0,0,48,1
2,3,740189,11811,164,33,0,9,1,0,0,0,10,2
3,4,2228301,47538,48,0,0,29,0,0,0,0,29,1
4,5,63749,982,2,0,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2434,10070,7594378,1690986,4965,1990,0,102,5,5,9,4,125,5
2435,100067,12354,428581,19,0,1,1,0,0,0,0,1,1
2436,100748,141755,207738,105,42,15,14,1,0,0,0,15,2
2437,100749,396556,8941,117,63,0,2,3,0,0,0,5,2


In [16]:
# Cuantos Clientes hay en cada Categoría de Score
last_year.groupby("Score").count().reset_index()

,Score,Clientes,ID_transaction,Importe_comp,DiasCalle_comp,DiasPago_comp,year_dummy,menos_15,menos_30,menos_45,menos_75,mas_75,Total
0,1,1579,1579,1579,1579,1579,1579,1579,1579,1579,1579,1579,1579
1,2,248,248,248,248,248,248,248,248,248,248,248,248
2,3,257,257,257,257,257,257,257,257,257,257,257,257
3,4,216,216,216,216,216,216,216,216,216,216,216,216
4,5,139,139,139,139,139,139,139,139,139,139,139,139


In [17]:
#Clientes con Diaspago de mas de 75, caso de estudio para validar utilidad del Score
last_year[last_year["mas_75"]>0]

,Clientes,ID_transaction,Importe_comp,DiasCalle_comp,DiasPago_comp,year_dummy,menos_15,menos_30,menos_45,menos_75,mas_75,Total,Score
16,22,1009672,78939,611,1843,0,8,8,0,0,1,17,4
20,26,2649122,419236,1644,814,0,18,15,3,3,2,41,5
22,28,1016584,10110,284,4757,0,9,0,0,0,4,13,5
29,36,451147,3569573,1525,1800,38,1,6,9,21,1,38,5
53,62,2486014,22876,1522,2329,0,34,0,0,0,1,35,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2194,2292,345462,2310121,826,1437,25,2,2,8,6,7,25,5
2200,2300,1316349,61325,1010,1116,0,17,0,0,0,1,18,4
2408,6956,3035105,328513,1759,1700,0,31,4,4,2,2,43,4
2430,9999,1531909,88062,1237,10011,0,14,3,0,0,5,22,5


In [18]:
#Dejando data lista para sumar al DF final
last_year = last_year.drop("ID_transaction", axis = 1)
last_year.columns = ["Clientes", "Importe_year", "DiasCalle_year", "DiasPago_year", "year_dummy_comp",
                     "menos_15", "menos_30", "menos_45", "menos_75", "mas_75", "Total_dias","Score_comp"]
last_year["year_dummy_comp"] = np.where(last_year["year_dummy_comp"]>0, 1, 0)
last_year

,Clientes,Importe_year,DiasCalle_year,DiasPago_year,year_dummy_comp,menos_15,menos_30,menos_45,menos_75,mas_75,Total_dias,Score_comp
0,1,149565,111,0,1,7,0,0,0,0,7,1
1,2,140191,708,0,0,48,0,0,0,0,48,1
2,3,11811,164,33,0,9,1,0,0,0,10,2
3,4,47538,48,0,0,29,0,0,0,0,29,1
4,5,982,2,0,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2434,10070,1690986,4965,1990,0,102,5,5,9,4,125,5
2435,100067,428581,19,0,1,1,0,0,0,0,1,1
2436,100748,207738,105,42,1,14,1,0,0,0,15,2
2437,100749,8941,117,63,0,2,3,0,0,0,5,2


In [19]:
#Calculando Categorias de pago segun la tabla DIASPAGO. Solo ultimo año
df_diaspago["Date"] = pd.to_datetime(df_diaspago["Date"])
desde = datetime.today() - timedelta(days=550)
pago_year = df_diaspago[df_diaspago["Date"]> desde]
pago_year["year_dummy_pago"] = 1


#Sumando aquellos que no hayan tenido transacciones el último año
clientes_pago = list(pago_year["Clientes"].unique())
clientes_pago = df_diaspago[~df_diaspago["Clientes"].isin(clientes_pago)]
clientes_pago["year_dummy_pago"] = 0
pago_year = pd.concat([pago_year, clientes_pago]) 


pago_year["menos_15"] = np.where(pago_year["DiasPago"]<15, 1 ,0)
pago_year["menos_30"] = np.where(
    (pago_year["DiasPago"]>=15) & (pago_year["DiasPago"]<30), 1 ,0)
pago_year["menos_45"] = np.where(
    (pago_year["DiasPago"]>=30) & (pago_year["DiasPago"]<45), 1 ,0)
pago_year["menos_75"] = np.where(
    (pago_year["DiasPago"]>=45) & (pago_year["DiasPago"]<75), 1 ,0)
pago_year["mas_75"] = np.where(pago_year["DiasPago"]>=75, 1 ,0)

pago_year = pago_year.groupby("Clientes").sum().reset_index()
pago_year["year_dummy_pago"] = np.where(pago_year["year_dummy_pago"]>0, 1 , 0)
pago_year

,Clientes,Importe_dp,DiasPago,Monto,ID_transaction,year_dummy_pago,menos_15,menos_30,menos_45,menos_75,mas_75
0,1,149563.78,0,149563.78,72953,1,7,0,0,0,0
1,2,140190.12,0,139593.89,4666395,0,74,0,0,0,0
2,3,21822.65,82,11256.00,1704721,0,19,2,1,0,0
3,4,47539.02,0,47539.02,2687326,0,35,0,0,0,0
4,5,982.10,0,982.10,63749,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2428,10070,2709602.98,9800,1709047.36,15095672,0,188,16,10,20,17
2429,100067,428581.00,0,428581.00,12354,1,1,0,0,0,0
2430,100748,275428.84,63,187993.49,223616,1,20,3,0,0,0
2431,100749,17203.01,89,8940.01,637149,0,4,4,0,0,0


In [20]:
#Calculo Score 
pago_year["Total"] = pago_year[["menos_15", "menos_30", "menos_45", "menos_75", "mas_75"]].sum(axis=1)
pago_year["Score_pago"] = 4
pago_year["Score_pago"] = np.where((pago_year["menos_15"]+pago_year["menos_30"]+pago_year["menos_45"])/
                              pago_year["Total"]>0.9, 3, pago_year["Score_pago"])
pago_year["Score_pago"] = np.where((pago_year["menos_15"]+pago_year["menos_30"])/pago_year["Total"]>0.9, 2, pago_year["Score_pago"])
pago_year["Score_pago"] = np.where(pago_year["menos_15"]/pago_year["Total"]>0.95, 1, pago_year["Score_pago"])
pago_year["Score_pago"] = np.where(pago_year["mas_75"]>0, pago_year["Score_pago"]+1, pago_year["Score_pago"])
pago_year

,Clientes,Importe_dp,DiasPago,Monto,ID_transaction,year_dummy_pago,menos_15,menos_30,menos_45,menos_75,mas_75,Total,Score_pago
0,1,149563.78,0,149563.78,72953,1,7,0,0,0,0,7,1
1,2,140190.12,0,139593.89,4666395,0,74,0,0,0,0,74,1
2,3,21822.65,82,11256.00,1704721,0,19,2,1,0,0,22,2
3,4,47539.02,0,47539.02,2687326,0,35,0,0,0,0,35,1
4,5,982.10,0,982.10,63749,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,10070,2709602.98,9800,1709047.36,15095672,0,188,16,10,20,17,251,5
2429,100067,428581.00,0,428581.00,12354,1,1,0,0,0,0,1,1
2430,100748,275428.84,63,187993.49,223616,1,20,3,0,0,0,23,2
2431,100749,17203.01,89,8940.01,637149,0,4,4,0,0,0,8,2


In [21]:
#Datos segun el dias pago para sumar al DF final
data_clientes = df_diaspago.groupby(['Clientes'])
counts_pagos = data_clientes.size().to_frame(name='counts')
counts_pagos = (counts_pagos
   .join(data_clientes.agg({'DiasPago': 'max'}).rename(columns={'DiasPago': 'DiasPago_max'}))
   .join(data_clientes.agg({'DiasPago': 'min'}).rename(columns={'DiasPago': 'DiasPago_min'}))
   .reset_index()
   )
counts_pagos

,Clientes,counts,DiasPago_max,DiasPago_min
0,1,19,0,0
1,2,74,0,0
2,3,22,38,0
3,4,35,0,0
4,5,1,0,0
...,...,...,...,...
2428,10070,251,989,0
2429,100067,1,0,0
2430,100748,120,44,0
2431,100749,8,25,0


In [22]:
#Juntando ambos Scores 
data = pd.merge(last_year, pago_year, on ="Clientes", how ="left")
data = data[[
    "Clientes", "Importe_year", "DiasCalle_year","year_dummy_comp" , "Total_dias", "Score_comp", 
    "DiasPago","year_dummy_pago","menos_15_y","menos_30_y","menos_45_y","menos_75_y","mas_75_y", "Total", "Score_pago"
]]
data

,Clientes,Importe_year,DiasCalle_year,year_dummy_comp,Total_dias,Score_comp,DiasPago,year_dummy_pago,menos_15_y,menos_30_y,menos_45_y,menos_75_y,mas_75_y,Total,Score_pago
0,1,149565,111,1,7,1,0.0,1.0,7.0,0.0,0.0,0.0,0.0,7.0,1.0
1,2,140191,708,0,48,1,0.0,0.0,74.0,0.0,0.0,0.0,0.0,74.0,1.0
2,3,11811,164,0,10,2,82.0,0.0,19.0,2.0,1.0,0.0,0.0,22.0,2.0
3,4,47538,48,0,29,1,0.0,0.0,35.0,0.0,0.0,0.0,0.0,35.0,1.0
4,5,982,2,0,1,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2434,10070,1690986,4965,0,125,5,9800.0,0.0,188.0,16.0,10.0,20.0,17.0,251.0,5.0
2435,100067,428581,19,1,1,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2436,100748,207738,105,1,15,2,63.0,1.0,20.0,3.0,0.0,0.0,0.0,23.0,2.0
2437,100749,8941,117,0,5,2,89.0,0.0,4.0,4.0,0.0,0.0,0.0,8.0,2.0


In [23]:
#Preparando data para sumar al DF final
borrador = df_completa.groupby("Clientes").mean().reset_index().drop("ID_transaction", axis = 1)
borrador.columns = ["Clientes", "ImporteMedio_h", "DiasCalle_h", "DiasPago_h"]

borrador = pd.merge(counts_pagos, borrador)
borrador

,Clientes,counts,DiasPago_max,DiasPago_min,ImporteMedio_h,DiasCalle_h,DiasPago_h
0,1,19,0,0,13570.411765,18.941176,0.000000
1,2,74,0,0,2920.645833,14.750000,0.000000
2,3,22,38,0,1181.100000,16.400000,3.300000
3,4,35,0,0,1639.241379,1.655172,0.000000
4,5,1,0,0,982.000000,2.000000,0.000000
...,...,...,...,...,...,...,...
2428,10070,251,989,0,13527.888000,39.720000,15.920000
2429,100067,1,0,0,428581.000000,19.000000,0.000000
2430,100748,120,44,0,6004.662338,24.207792,5.051948
2431,100749,8,25,0,1788.200000,23.400000,12.600000


In [24]:
#Juntando toda la data
data = pd.merge(borrador, data , how = "left", on = "Clientes")
data

,Clientes,counts,DiasPago_max,DiasPago_min,ImporteMedio_h,DiasCalle_h,DiasPago_h,Importe_year,DiasCalle_year,year_dummy_comp,...,Score_comp,DiasPago,year_dummy_pago,menos_15_y,menos_30_y,menos_45_y,menos_75_y,mas_75_y,Total,Score_pago
0,1,19,0,0,13570.411765,18.941176,0.000000,149565,111,1,...,1,0.0,1.0,7.0,0.0,0.0,0.0,0.0,7.0,1.0
1,2,74,0,0,2920.645833,14.750000,0.000000,140191,708,0,...,1,0.0,0.0,74.0,0.0,0.0,0.0,0.0,74.0,1.0
2,3,22,38,0,1181.100000,16.400000,3.300000,11811,164,0,...,2,82.0,0.0,19.0,2.0,1.0,0.0,0.0,22.0,2.0
3,4,35,0,0,1639.241379,1.655172,0.000000,47538,48,0,...,1,0.0,0.0,35.0,0.0,0.0,0.0,0.0,35.0,1.0
4,5,1,0,0,982.000000,2.000000,0.000000,982,2,0,...,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2428,10070,251,989,0,13527.888000,39.720000,15.920000,1690986,4965,0,...,5,9800.0,0.0,188.0,16.0,10.0,20.0,17.0,251.0,5.0
2429,100067,1,0,0,428581.000000,19.000000,0.000000,428581,19,1,...,1,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
2430,100748,120,44,0,6004.662338,24.207792,5.051948,207738,105,1,...,2,63.0,1.0,20.0,3.0,0.0,0.0,0.0,23.0,2.0
2431,100749,8,25,0,1788.200000,23.400000,12.600000,8941,117,0,...,2,89.0,0.0,4.0,4.0,0.0,0.0,0.0,8.0,2.0


In [25]:
#Exportando data 
data.to_csv("Data_pago_score.csv",index=False)

In [26]:
#Replicando para Dias Calle:

df_completa["Date"] = pd.to_datetime(df_completa["Date"])
desde = datetime.today() - timedelta(days=550)
last_year = df_completa[df_completa["Date"]> desde]
last_year["year_dummy"] = 1
clientes_completa = list(last_year["Clientes"].unique())
clientes_completa = df_completa[~df_completa["Clientes"].isin(clientes_completa)]
clientes_completa["year_dummy"] = 0
last_year = pd.concat([last_year, clientes_completa]) 

last_year["menos_15"] = np.where(last_year["DiasCalle_comp"]<15, 1 ,0)
last_year["menos_30"] = np.where(
    (last_year["DiasCalle_comp"]>=15) & (last_year["DiasCalle_comp"]<30), 1 ,0)
last_year["menos_45"] = np.where(
    (last_year["DiasCalle_comp"]>=30) & (last_year["DiasCalle_comp"]<45), 1 ,0)
last_year["menos_75"] = np.where(
    (last_year["DiasCalle_comp"]>=45) & (last_year["DiasCalle_comp"]<75), 1 ,0)
last_year["mas_75"] = np.where(last_year["DiasCalle_comp"]>=75, 1 ,0)

last_year = last_year.groupby("Clientes").sum().reset_index()

last_year["Total"] = last_year[["menos_15", "menos_30", "menos_45", "menos_75", "mas_75"]].sum(axis=1)
last_year["Score"] = 4
last_year["Score"] = np.where((last_year["menos_15"]+last_year["menos_30"]+last_year["menos_45"])/
                              last_year["Total"]>0.9, 3, last_year["Score"])
last_year["Score"] = np.where((last_year["menos_15"]+last_year["menos_30"])/last_year["Total"]>0.95, 2, last_year["Score"])
last_year["Score"] = np.where(last_year["menos_15"]/last_year["Total"]>0.95, 1, last_year["Score"])
last_year["Score"] = np.where(last_year["mas_75"]>0, last_year["Score"]+1, last_year["Score"])

last_year = last_year.drop("ID_transaction", axis = 1)
last_year.columns = ["Clientes", "Importe_year", "DiasCalle_year", "DiasPago_year", "year_dummy_comp",
                     "menos_15", "menos_30", "menos_45", "menos_75", "mas_75", "Total_dias","Score_comp"]
last_year["year_dummy_comp"] = np.where(last_year["year_dummy_comp"]>0, 1, 0)
last_year

,Clientes,Importe_year,DiasCalle_year,DiasPago_year,year_dummy_comp,menos_15,menos_30,menos_45,menos_75,mas_75,Total_dias,Score_comp
0,1,149565,111,0,1,3,3,1,0,0,7,3
1,2,140191,708,0,0,40,4,1,2,1,48,4
2,3,11811,164,33,0,7,0,2,0,1,10,5
3,4,47538,48,0,0,29,0,0,0,0,29,1
4,5,982,2,0,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2434,10070,1690986,4965,1990,0,22,8,44,45,6,125,5
2435,100067,428581,19,0,1,0,1,0,0,0,1,2
2436,100748,207738,105,42,1,12,3,0,0,0,15,2
2437,100749,8941,117,63,0,2,0,3,0,0,5,3


In [27]:
#Calculando Categorias de pago segun la tabla DIASCALLE. Solo ultimo año
df_diascalle["Date"] = pd.to_datetime(df_diascalle["Date"])
desde = datetime.today() - timedelta(days=550)
pago_year = df_diascalle[df_diascalle["Date"]> desde]
pago_year["year_dummy_pago"] = 1


#Sumando aquellos que no hayan tenido transacciones el último año
clientes_pago = list(pago_year["Clientes"].unique())
clientes_pago = df_diascalle[~df_diascalle["Clientes"].isin(clientes_pago)]
clientes_pago["year_dummy_pago"] = 0
pago_year = pd.concat([pago_year, clientes_pago]) 

pago_year["menos_15"] = np.where(pago_year["DiasCalle"]<15, 1 ,0)
pago_year["menos_30"] = np.where(
    (pago_year["DiasCalle"]>=15) & (pago_year["DiasCalle"]<30), 1 ,0)
pago_year["menos_45"] = np.where(
    (pago_year["DiasCalle"]>=30) & (pago_year["DiasCalle"]<45), 1 ,0)
pago_year["menos_75"] = np.where(
    (pago_year["DiasCalle"]>=45) & (pago_year["DiasCalle"]<75), 1 ,0)
pago_year["mas_75"] = np.where(pago_year["DiasCalle"]>=75, 1 ,0)

pago_year = pago_year.groupby("Clientes").sum().reset_index()
pago_year["year_dummy_pago"] = np.where(pago_year["year_dummy_pago"]>0, 1 , 0)

pago_year["Total"] = pago_year[["menos_15", "menos_30", "menos_45", "menos_75", "mas_75"]].sum(axis=1)
pago_year["Score_pago"] = 4
pago_year["Score_pago"] = np.where((pago_year["menos_15"]+pago_year["menos_30"]+pago_year["menos_45"])/
                              pago_year["Total"]>0.9, 3, pago_year["Score_pago"])
pago_year["Score_pago"] = np.where((pago_year["menos_15"]+pago_year["menos_30"])/pago_year["Total"]>0.9, 2, pago_year["Score_pago"])
pago_year["Score_pago"] = np.where(pago_year["menos_15"]/pago_year["Total"]>0.95, 1, pago_year["Score_pago"])
pago_year["Score_pago"] = np.where(pago_year["mas_75"]>0, pago_year["Score_pago"]+1, pago_year["Score_pago"])
pago_year

,Clientes,Importe_dc,Importe_comprobante,DiasCalle,year_dummy_pago,menos_15,menos_30,menos_45,menos_75,mas_75,Total,Score_pago
0,1,252565.89,148558.78,160,1,6,4,0,1,0,11,2
1,2,173682.02,142543.21,2374,0,72,6,0,0,5,83,3
2,3,11811.00,11811.00,270,0,9,2,4,0,1,16,4
3,4,56941.09,47814.09,82,0,40,1,0,0,0,41,1
4,5,982.10,982.10,2,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2455,10070,8111123.98,1680769.23,22673,0,82,66,155,167,58,528,5
2456,100067,857162.00,428582.00,37,1,0,2,0,0,0,2,2
2457,100748,220538.74,207738.49,136,1,17,4,0,0,0,21,2
2458,100749,18986.43,12034.97,1070,0,2,0,4,0,2,8,5


In [28]:
data_clientes = df_diascalle.groupby(['Clientes'])
counts_pagos = data_clientes.size().to_frame(name='counts')
counts_pagos = (counts_pagos
   .join(data_clientes.agg({'DiasCalle': 'max'}).rename(columns={'DiasCalle': 'DiasCalle_max'}))
   .join(data_clientes.agg({'DiasCalle': 'min'}).rename(columns={'DiasCalle': 'DiasCalle_min'}))
   .reset_index()
   )

borrador = df_completa.groupby("Clientes").mean().reset_index().drop("ID_transaction", axis = 1)
borrador.columns = ["Clientes", "ImporteMedio_h", "DiasCalle_h", "DiasPago_h"]

borrador = pd.merge(counts_pagos, borrador)
borrador

,Clientes,counts,DiasCalle_max,DiasCalle_min,ImporteMedio_h,DiasCalle_h,DiasPago_h
0,1,31,253,2,13570.411765,18.941176,0.000000
1,2,83,602,3,2920.645833,14.750000,0.000000
2,3,16,78,0,1181.100000,16.400000,3.300000
3,4,41,23,0,1639.241379,1.655172,0.000000
4,5,1,2,2,982.000000,2.000000,0.000000
...,...,...,...,...,...,...,...
2434,10070,528,161,0,13527.888000,39.720000,15.920000
2435,100067,2,20,17,428581.000000,19.000000,0.000000
2436,100748,157,78,0,6004.662338,24.207792,5.051948
2437,100749,8,473,0,1788.200000,23.400000,12.600000


In [29]:
data = pd.merge(last_year, pago_year, on ="Clientes", how ="left")
data = data[[
    "Clientes", "Importe_year", "DiasCalle_year","year_dummy_comp" , "Total_dias", "Score_comp", 
    "DiasCalle","year_dummy_pago","menos_15_y","menos_30_y","menos_45_y","menos_75_y","mas_75_y", "Total", "Score_pago"
]]
data = pd.merge(borrador, data , how = "left", on = "Clientes")
data

,Clientes,counts,DiasCalle_max,DiasCalle_min,ImporteMedio_h,DiasCalle_h,DiasPago_h,Importe_year,DiasCalle_year,year_dummy_comp,...,Score_comp,DiasCalle,year_dummy_pago,menos_15_y,menos_30_y,menos_45_y,menos_75_y,mas_75_y,Total,Score_pago
0,1,31,253,2,13570.411765,18.941176,0.000000,149565,111,1,...,3,160,1,6,4,0,1,0,11,2
1,2,83,602,3,2920.645833,14.750000,0.000000,140191,708,0,...,4,2374,0,72,6,0,0,5,83,3
2,3,16,78,0,1181.100000,16.400000,3.300000,11811,164,0,...,5,270,0,9,2,4,0,1,16,4
3,4,41,23,0,1639.241379,1.655172,0.000000,47538,48,0,...,1,82,0,40,1,0,0,0,41,1
4,5,1,2,2,982.000000,2.000000,0.000000,982,2,0,...,1,2,0,1,0,0,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2434,10070,528,161,0,13527.888000,39.720000,15.920000,1690986,4965,0,...,5,22673,0,82,66,155,167,58,528,5
2435,100067,2,20,17,428581.000000,19.000000,0.000000,428581,19,1,...,2,37,1,0,2,0,0,0,2,2
2436,100748,157,78,0,6004.662338,24.207792,5.051948,207738,105,1,...,2,136,1,17,4,0,0,0,21,2
2437,100749,8,473,0,1788.200000,23.400000,12.600000,8941,117,0,...,3,1070,0,2,0,4,0,2,8,5


In [30]:
#Exportando data 
data.to_csv("Data_calle_score.csv",index=False)